In [1]:
import os
try:
    from google.colab import drive
    COLAB = True
    print("Estoy en Google CoLab")
    %tensorflow_version 2.x
    !pip install livelossplot
except:
    os.environ["CUDA_VISIBLE_DEVICES"]="0" 
    print("No estoy en Google CoLab")
    COLAB = False

Estoy en Google CoLab
     |████████████████████████████████| 793 kB 5.8 MB/s 
     |████████████████████████████████| 381 kB 43.9 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.


# Autoencodificadores variacionales

In [2]:
# Cargamos las librerías necesarias para la práctica
from keras import models
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, Flatten, LeakyReLU, Activation, Dropout
from keras.layers import Layer, Reshape, Conv2DTranspose, Lambda, ReLU, BatchNormalization
from keras.metrics import binary_crossentropy
from keras.optimizers import Adam
try:
  from livelossplot import PlotLossesKerasTF
except:
  !pip install livelossplot
  from livelossplot import PlotLossesKerasTF
    
try:
  import tqdm
except:
  !pip install tqdm
  import tqdm
    
from keras.utils import plot_model
from keras.datasets import mnist, cifar10
from tqdm import tqdm
from io import BytesIO, StringIO
import numpy as np
import pathlib
import cv2
import matplotlib
import os
import random
import re
import tensorflow as tf
# %matplotlib inline
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (15,5) # Para el tamaño de la imagen
matplotlib.rcParams['figure.figsize'] = (15,5)

import seaborn as sns
import pandas as pd
from utils import *

ImportError: ignored

In [ ]:
# MNIST dataset
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

image_size = x_train_mnist.shape[1]
x_train_mnist = np.reshape(x_train_mnist, [-1, image_size, image_size, 1])
x_test_mnist = np.reshape(x_test_mnist, [-1, image_size, image_size, 1])
x_train_mnist = x_train_mnist.astype('float32') / 255
x_test_mnist = x_test_mnist.astype('float32') / 255

## Autoencoders variacionales

El espacio latente generado por los autoencoders sufren de falta de simetría, las muestras no se distribuyen de manera equitativa en el espacio y hay mucho espacios vacios. Esto provoca que el generador genere en gran parte del espacio imágenes irreconocibles. Los autoencoders variacionales intentan solventar estas deficiencias. 

<img src='https://hdvirtual.us.es/discovirt/index.php/s/NE9rr6KH9CTMSqN/preview' width='70%' />

Para ello el vector latente se mapeará de una distribución normal multivariante alrededor de un punto en el espacio latente:

$$z = \mu + \sigma\epsilon$$

$z$ es nuestro vector latente, que se genera a partir de $\mu$ y $\sigma$ que deberá aprender el autoencoder. $\epsilon$ se muestrea de una distribució nnormal estándar. 

Como vemos en el gráfico anterior, tendremos que modificar la codificador, pero el decodificador sigue manteniendo la misma estructura, así que vamos a partir de nuestra clase `Autoencoder` para implementa una nueva clase `VariatioalAutoencoder`

Otro elemento que cambia en los autoencoders variacionales es funcion de coste. Ahora sera:

$$VAE_{loss} = \eta_1 METRICA + \eta_2 D_{KL}$$

donde:

$$D_{KL} = \frac{1}{2} \sum^k_i (1 + log(\sigma_i^2) - \mu_i^2 - \sigma_i^2)$$

y *METRICA* puede ser *MSE*, *Binary corss entropy*, *SSIM*, .... 

In [ ]:
class VariationalAutoencoder(Autoencoder):
    
    def __init__(self, eta=1, alpha=0.001, loss_type='cross_entropy', **kwargs):
        super(VariationalAutoencoder, self).__init__(**kwargs)
        
        self.eta = eta
        self.alpha = alpha
        self.loss_type = loss_type
        
        self.total_loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")
        
    def _create_latent_vector(self, encoder_input, x):
        
        self._mu = Dense(self.z_dim, name='mu')(x)                  #(1)                        
        self._log_var = Dense(self.z_dim, name='log_var')(x)
        
        self._encoder_mu_log_var = Model(encoder_input, (self._mu, self._log_var))    #(2)
        
        def sampling(args):
            mu, log_var = args
            epsilon = K.random_normal(shape=K.shape(mu), mean=0., stddev=1.)
            return mu + K.exp(log_var / 2) * epsilon
        
        self.encoder_output = Lambda(sampling, name='encoder_output')([self._mu, self._log_var])   #(3)

        return self.encoder_output
    
    def call(self, data):
        (z_mean, z_log_var), z = self._encoder_mu_log_var(data), self.encoder(data)
        reconstruction = self.decoder(z)
        data = data
        
        if self.loss_type == 'cross_entropy':
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
        elif self.loss_type == 'ssim':
            reconstruction_loss = 1 - tf.reduce_mean(tf.image.ssim(data, reconstruction, 2.0))

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
        total_loss = self.eta * reconstruction_loss +  self.alpha * kl_loss
        
        # actualizamos las métricas
        self.add_metric(kl_loss, name='kl_loss', aggregation='mean')
        self.add_metric(total_loss, name='loss', aggregation='mean')
        self.add_metric(reconstruction_loss, name='reconstruction_loss', aggregation='mean')
        return reconstruction
    
    def train_step(self, data):

        with tf.GradientTape() as tape:
            (z_mean, z_log_var), z = self._encoder_mu_log_var(data), self.encoder(data)
            reconstruction = self.decoder(z)
            data = data[0]

            if self.loss_type == 'cross_entropy':
                reconstruction_loss = tf.reduce_mean(
                    tf.reduce_sum(
                        keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                    )
                )
            elif self.loss_type == 'ssim':
                reconstruction_loss = 1 - tf.reduce_mean(tf.image.ssim(data, reconstruction, 2.0))

            
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = self.eta * reconstruction_loss +  self.alpha * kl_loss
            
        # obtenemos los gradientes        
        grads = tape.gradient(total_loss, self.trainable_weights)
        
        # actualizamos los pesos
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # actualizamos las métricas
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

   1. En vez de conectar directamente el vector aplanado al espacio latente se conecta a dos capas densas representante de $\mu$ y $\sigma$. Guardamos las capas en atributos de la clase porqué nos hará falta para computar la función de coste.
   2. Este modelo Keras auxiliar nos sirve para obtener $\mu$ y $\sigma$ a partir de una imagen de entrada.
   3. La capa lambda usa la función `sampling` para muestrear de la distribución definida por $\mu$ y $\sigma$.
   
Para comparar la diferencia, nos vamos a crear un autoencoder variacional con la misma estructura base que en el ejemplo del autoencoder para MNIST:

In [ ]:
VAE_MNIST = VariationalAutoencoder(
        eta = 1,
        alpha = 1/50,
        loss_type='ssim',
         input_dim = (28,28,1)
        , encoder_conv_filters = [32,64,64,64]
        , encoder_conv_kernel_size = [3,3,3,3]
        , encoder_conv_strides = [1,2,2,1]
        , decoder_conv_filters = [64,64,32,1]
        , decoder_conv_kernel_size = [3,3,3,3]
        , decoder_conv_strides = [1,2,2,1]
        , z_dim = 2
        , activation = 'relu')       

Vamos a ver que se corresponde con la estructura esperada usando el método `plot_model` de Keras:

In [ ]:
plot_model(VAE_MNIST.model, show_shapes=True, show_layer_names=True)

Vamos a entrenar nuestra VAE para comparar el espacio latente generado:

In [ ]:
VAE_MNIST.compile(optimizer=Adam(0.0005))

VAE_MNIST.fit(x=x_train_mnist, 
             validation_data=(x_test_mnist, x_test_mnist),
             batch_size=32, shuffle=True, epochs=25,
             callbacks=[PlotLossesKerasTF()])

In [ ]:
VAE_MNIST.model.save_weights('vae_mnist_weights.h5')

Vamos a mostrar ahora nuestro espacio latente:

In [ ]:
import utils as u
u.show_latent_space(VAE_MNIST, x_test_mnist, y_test_mnist)

In [ ]:
u.show_latent_space(VAE_MNIST, x_test_mnist, y_test_mnist)


Si nos centramos más en $0,0$ y en una una desviación de aproximadamente 1 veremos como hemos mejorando nuestro espacios latente:

Como en el caso anterior podemos hacer un muestreo de nuestro espacio latente. Dado que hemos hemos *obligado* a nuestro autoencoder a que el espacio esté centrado en 0 y tengan una deviación de 1 aproximadamente, podemos muestrear unicamente en el rango -1, 1:

In [ ]:
sample_latent_space(VAE_MNIST, x_range=(-1.3,1.3), y_range=(-1.3,1.3))

## Usando autoencoders variacionales para generar caras

En este ejercicio usaremos el conjunto de datos [CelebFaces Attributes (CelebA)](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) para entrenar nuestro siguiente autoencoder. Este conjunto de datos compuesto por aproximadamente 200.000 imágenes de caras de famosos, cada una de ellas anotadas con una o más etiquetas del tipo: sonriendo, con gafas, llevando sombrero, etc...

<img src='https://hdvirtual.us.es/discovirt/index.php/s/yysHBNpofeGKdyG/download' width='70%' />

No vamos a usar dichas etiquetas para entrenar nuestra VAE, pero nos serán de utilidad para llevar a cabo ciertas operaciones en el espacio latente.

En primer lugar debemos descargar y descomprimir el conjunto de datos:

In [ ]:
data_path = tf.keras.utils.get_file(
    "celeba.tar.gz",
    "https://hdvirtual.us.es/discovirt/index.php/s/9KMEFpajdzgDAC9/download",
    extract=True,
    archive_format='tar'
)

DATA_FOLDER = pathlib.Path(data_path).parent / 'celeba' 

IMG_FOLDER = os.path.join(DATA_FOLDER, 'images')
att_file = os.path.join(DATA_FOLDER, 'att.csv')

# obtenemos los nombres de los archivos de imágenes
file_names = os.listdir(IMG_FOLDER)

print('Total de imagenes: %d' % len(file_names))

Lo siguiente será crear nuestro VAE. Trabajamos con imágenes en color por lo que la entrada y la salida tendrán 3 canales. En este caso vamos a normalizar la salida de cada capa convolucional y vamos a aplicar un dropout del 25%.

El resto es prácticamente igual a los ejemplos anteriores:

In [ ]:
INPUT_DIM = (128, 128, 3)

VAE_CELEBA = VariationalAutoencoder(
                        eta = 1,
                        alpha = 1e-3,
                        loss_type='ssim',
                        input_dim = INPUT_DIM
                        , encoder_conv_filters=[32,64,64, 64]
                        , encoder_conv_kernel_size=[3,3,3,3]
                        , encoder_conv_strides=[2,2,2,2]
                        , decoder_conv_filters=[64,64,32,3]
                        , decoder_conv_kernel_size=[3,3,3,3]
                        , decoder_conv_strides=[2,2,2,2]
                        , activation = ReLU
                        , z_dim=200
                        , use_batch_normalization=True
                        , dropout=0.25)

VAE_CELEBA.compile(optimizer=Adam(0.0005))

Este conjunto de datos tiene un tamaño total de 1.3GB en disco. El problema viene cuando convertimos la imágenes a array numpy de floats. El tamaño se multiplica llegando a pesar más de 30GB o más. Dado que no podemos cargar en memoria todas las imágenes, Keras nos da herramientas para trabajar con alimentación dinámica. Esto es, vamos a ir leyendo de disco los batches conforme vayan haciendo falta.

El procedimiento es muy sencillo. Usamos la clase `ImageDataGenerator` indicando el escalado y el porcentaje del conjunto de datos que queremos reservar para validación. Con el método `flow_from_directory` indicamos la carpeta donde se encuentran las imágenes, el tamaño del batch y con el atributo `subset` establecemos si ese flujo será de la parte reservada para entrenamietno o para validación. Por último tenemos que entrenar, que ahora lo haremos con el método `fit_generator`.

```
from keras_preprocessing.image import ImageDataGenerator

INPUT_DIM = (128,128,3)
BATCH_SIZE = 128


# creamos nuestro iterador indicando que tras leer la imagen se debe escalar dividiendo por 255
data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

train_data_flow = data_gen.flow_from_directory(
    DATA_FOLDER, 
    target_size = INPUT_DIM[:2], 
    batch_size = BATCH_SIZE, 
    shuffle = True, 
    class_mode = 'input', 
    subset = "training")

val_data_flow = data_gen.flow_from_directory(
    DATA_FOLDER, 
    target_size = INPUT_DIM[:2], 
    batch_size = BATCH_SIZE, 
    shuffle = True, 
    class_mode = 'input', 
    subset = "validation")

steps_per_epoch = int((len(file_names) * 0.7) / BATCH_SIZE)

VAE_CELEBA.model.fit_generator(train_data_flow, validation_data=val_data_flow, shuffle = True, 
                               epochs = 200, callbacks = [PlotLossesKeras()], 
                               steps_per_epoch=steps_per_epoch 
    )
```

Con este código cada época tardaría unos 3min. Si vamos a ejecutar 200 epocas, no llevaría el entrenamiento 600 min sin contar la evaluación. 

Recordamos que el conjunto de datos en disco ocupa 1.3GB. ¿Y si lo precargamos en memoria en binario? De esa forma evitamos la lectura a disco de los mismo archivos en cada época y dado que son solo 1.3GB no deberíamos tener muchos problemas de memoria. 

Este procedimiento los podemos hacer con una sola línea de código:

In [ ]:
X = [open(os.path.join(DATA_FOLDER, 'images', f), 'rb').read() for f in tqdm(file_names)]

Una vez leidos los archivos en memoria tendriamos que ir convirtiendolos a arrays de numpy. Vamos a ver cuando tardaríamos:

In [ ]:
import imageio
for im in tqdm(X):
    b = BytesIO(im)
    file_bytes = np.asarray(bytearray(BytesIO(im).read()), dtype=np.uint8)
    im = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)



Se tardaría 1 min y 21 seg aproximadamente. Por lo que estariamos reduciendo un 50% el tiempo de procesamiento.

Para aprovechar este truco tenemos que crearnos una clase que haga de iterador, como la clase `ImageDataGenerator` de Keras.

La implementación de clase sería la siguiente:

In [ ]:
class CelebaInMemoryIterator(tf.keras.utils.Sequence):
       
    def __init__(self, im_bytes, batch_size, shuffle=True):
        self.im_bytes = im_bytes
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.__len = int(len(self.im_bytes) / self.batch_size)
        
        self.reset()

    def __len__(self):
        return self.__len

    def __getitem__(self, index):
        # comienzo y final del batch
        i = index * self.batch_size
        j = i + self.batch_size
        
        # indixes del batch
        indexes = self.__indexes[i:j]
        
        # imagenes binarias del batch
        batch = [self.im_bytes[k] for k in indexes]

        # lista final de arrays
        ims = []
   
        # convertimos a numpy arrays
        for im in batch:
            b = BytesIO(im)
            file_bytes = np.asarray(bytearray(BytesIO(im).read()), dtype=np.uint8)
            im = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (128, 128)) 
            ims.append(im)

        # escalamos a valores entre 0 y 1
        ims = np.array(ims).astype('float') / 255
        
        
        return ims, ims


    def on_eponch_end(self):
        self.reset()
        
    def reset(self):
        self.__indexes = list(range(len(self.im_bytes)))           
        if self.shuffle:
            self.__indexes = random.sample(self.__indexes, len(self.__indexes))
        self.__current_index = 0

    def __iter__(self):
        return self

    def __next__(self, *args, **kwargs):
        # si hemos llegado al final reiniciamos
        if self.__current_index >= self.__len:
            self.reset()
            
        # cogemos el siguiente batch
        r =  self.__getitem__(self.__current_index)
        
        # incrementamos el indice
        self.__current_index += 1
        
        
        return r
    

El método más importante es el método `__getitem__`. Este método recibe un parámetro `index` que correcto con el número de batch de nuestro conjunto de datos. Con ese número se debe preparar el batch y devolverlo.

El método `reset` simplemente reinicia a 0 el indice y baraja los indices si el atributo `shuffle` se estableció a true. Por último el métod `next` devuelve el siguiente batch a ser procesado. Básicamente controla si es necesario reiniciar el índice y lo incrementa en cada llamada.

Ahora podemos usar nuestra clase con `fit_generator`:

In [ ]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 128

# dividimos el conjunto de datos entrenamiento y test
indexes = list(range(len(X)))
random.shuffle(indexes)
cut_point = int(len(X) * 0.7)
X_train = [X[i] for i in indexes[:cut_point]]
X_test = [X[i] for i in indexes[cut_point:]]    

# creamos los iteradores
train_generator = CelebaInMemoryIterator(X_train, 32)
test_generator = CelebaInMemoryIterator(X_test, 128)

# entrenamos usando los iteradores
VAE_CELEBA.fit(train_generator, validation_data=test_generator, 
                               epochs = 10, callbacks = [PlotLossesKerasTF()])
        


Vamos a ver la calidad de la reconstrucción.

In [ ]:
predict_generator = CelebaInMemoryIterator(X_test[:10], 10, shuffle=True)
x, _ = predict_generator.__next__()
draw2compare(x, VAE_CELEBA.model.predict(x))

Se aprecia que la VAE ha capturado elementos clave de la cara (el ángulo de la cara, el peinado, la expresión, etc.) y que los detalles más finos se pierden. Aun así, es importante recordar que el objetivo no es realizar una reconstrucción perfecta (aunque sería deseable). El objetivo principal es poder hacer un muestreo en el espacio latente.

Vamos a realizar un muestreo del espacio:

In [ ]:
rows = 3
cols = 10
y = tuple([VAE_CELEBA.decoder.predict(np.random.normal(size=(cols, VAE_CELEBA.z_dim))) for _ in range(rows)])
len(y), len(y[0])

In [ ]:
draw2compare(*y)

Es sorprendente como la VAE es capaz de crear imágenes de personas a partir de puntos tomados aleatoriamente.

### Aritmética en el espacio latente

Un beneficio de las representaciones en el espacio latente vectorial es que podemos llevar a cabo operaciones vectorailes y después decodificar el resultado de dicha operación.

Por ejemplo, supongamos que queremos hacer que una imagen de una alguien que tiene una expesión de tristeza la queremos convertir en sonriente. Para poder hacer esto necesitamos un vector latente que apunte en la dirección del espacio en el cual se incrementa la sonrisa. Si agregamos este vector a otro vector latente obtendremos un vector cuya imagen asociada será más sonriente que la orignal.

¿Como encontramos esos vectores direccionales? Para ello vamos a necesitar el conjunto de datos atributos. Cada imagen tiene asociada un conjunto de atributos, en ellos uno de es si esta sonriendo o no. Si calculamos la media de los vectores latentes de imágenes sonrientes y le restamos la media de los vectores latentes de imágnes no sonrientes obtedremos nuestro vector direccional que apunta desde *no sonriente* a *sonriente*.

El formato de este archivo un es poco especial y tenemos que hacerle unas modficaciones antes de poder leerlo. Un vez lo hemos leido vamos a reordenarlo que siga el mismo orden que nuestra lista de ficheros:

In [ ]:
# abrimos el archivo en modo lectura
with open(att_file, 'r') as fp:
    # eliminamos la primera línea, es el número de imágene el dataset (202599)
    lines = fp.readlines()[1:]
    # la siguiente linea es el nombre de cada columna  pero hay un espacio que demos eliminar y agregar
    # una columna al principio para el nombre del fichero de imagen
    lines[0] = ('File_Name ' + lines[0]).strip() + '\n'
    # por último sustituimos los espacios por comas
    lines = [re.sub('[ ]+', ',', l) for l in lines]
    
# leemos (parseamos) con pandas usando StringIO ya que tenemos el archivo en memoria
df_data = pd.read_csv(StringIO(''.join(lines)))

# reordenamos el conjunto de datos para que tenga el mismo orden que nuestra lista de ficheros
df_data = df_data.set_index('File_Name').loc[file_names].reset_index()

# mostramos las primreas filas del conjunto de datos
df_data.head()

Como vemos tenemos una ficha por cada fichero identificado por su nombre. Cada columna corresponde a una caracterísca de esa imagen. Si la imagen tiene ese atributo está marcado con un 1, si no lo tiene está marcado con -1.

Lo primero que vamos a hacer es seleccionar un atributo y calcular el vector medio de todos los vectores latentes que tienen ese atributo y los que no lo tienen. Para ello primero debemos obtener todos los vectores latentes. Vamos a usar nuestro iterador para ir obteniendo los vectores latentes e ir computando la media batch a batch:

In [ ]:
# vamos a probar con el atributo smiling
attribute = 'Smiling'

# tomamos los índices de que aquellas imágenes marcadas como 'soriendo' y aquellas que no
with_att_files = df_data[df_data[attribute] == 1].index
without_att_files = df_data[df_data[attribute] == -1].index

# creamos dos listas, una con los binarios de las imagnes marcadas como 'sonriendo' y la que no
X_with_att = [X[i] for i in with_att_files]
X_without_att = [X[i] for i in without_att_files]

def get_mean_vector(generator):
    
    mean_vector = None
    # usamos el generador para ir recorriendo los batches
    for i in tqdm(range(len(generator))):
        # obtenemos los vectors latentes del batch
        pred = VAE_CELEBA.encoder.predict(generator.__next__()[0])
        
        # si el primero calculamos la media de ese batch
        # en otro caso calculamos la media de la media de ese batch con la anterior
        if mean_vector is None:
            mean_vector = pred.mean(axis=0)
        else:
            mean_vector += pred.mean(axis=0)
            mean_vector /= 2
            
    return mean_vector

# obtenenmos el vector medio de las imágenes marcadas como 'sonriendo'
predict_generator = CelebaInMemoryIterator(X_with_att, 128, shuffle=False)
with_att_mean_vector = get_mean_vector(predict_generator)

# obtenenmos el vector medio de las imágenes marcadas como no 'sonriendo'
predict_generator = CelebaInMemoryIterator(X_without_att, 128, shuffle=False)
without_att_mean_vector = get_mean_vector(predict_generator)

Vamos a visualizar esos dos vectores del espacio latente:

In [ ]:
draw2compare(VAE_CELEBA.decoder.predict(np.array([with_att_mean_vector, without_att_mean_vector])))

Y ahora ya podemos calcular nuestro vector directional desde *no sonriente* a *sonriente*:

In [ ]:
directional_vector = with_att_mean_vector - without_att_mean_vector

El siguiente paso es simplemente tomar una imagen de nuestro conjunto de datos e ir sumandole o restandole nuestro vector direccional y ver el efecto:

In [ ]:
# obtenemos los vectores de 10 imágenes al azar
sample = CelebaInMemoryIterator(X, 10, shuffle=True).__next__()[0]
vectors = VAE_CELEBA.encoder.predict(sample)

# esta será la proporción del atributo que añadiremos a nuestros vectores
factors = [-4,-3,-2,-1,0,1,2,3,4]

# computamos la decoficación de los vectores modificador: v + f * d
changes = np.array([VAE_CELEBA.decoder.predict(vectors + f * directional_vector) for f in factors])

# mostramos
draw2compare(*list(changes))

Es sumamente interesante que aunque nos estamos alejando en el espacio latente desde el punto orignal la mayor parte de imagen no cambia excepto en lo que concierne a la característica que estamos modificando. Esto pone de manifiesto el poder de los autoencoders variacionales para capturar y permitirnos manipular características de alto nivel de abstracción.

Otra operación interesante que podemos llevar a cabo es la morphing. Esta consiste en tomar dos imágenes A y B y trazar una línea recta entre ambas en el espacio latente. Moviendomos a lo largo de esta línea podemos recrear la progresiva transformación de una cara en la otra:

Matemáticamente:

$$
z_{new} = (1 - \alpha) z_A - \alpha z_B
$$

donde $\alpha$ es un número entre 0 y 1 que determina en que posición entre el punto A y B nos encontramos (o que aportación del vector A y el vector B hay en el vector resultante).

In [ ]:
# Tomamos una lista de vectores A y vectores B
sample = CelebaInMemoryIterator(X, 10, shuffle=True).__next__()[0]
vectors_A = VAE_CELEBA.encoder.predict(sample)

sample = CelebaInMemoryIterator(X, 10, shuffle=True).__next__()[0]
vectors_B = VAE_CELEBA.encoder.predict(sample)

images = [VAE_CELEBA.decoder.predict(vectors_A)]
for alpha in np.arange(0,1,0.1)[1:]:
    z_new = VAE_CELEBA.decoder.predict((1-alpha)*vectors_A + alpha*vectors_B)
    images.append(z_new)
images.append(VAE_CELEBA.decoder.predict(vectors_B))

# mostramos
draw2compare(*images)



La transición se realiza suavemente incluiso cuando se están alterando varias características simultaneamente gracias a las continuidad del espacio latente.

# Referencias

 - Generative Deep Learning. Teaching Machines to Paint, Write, Compose and Play. David Foster. O'Reilly
 - Advanced Deep Learning with Keras. Rowel Atienza. Packt.